In [2]:
import googlemaps
import pprint 
import os
from postgres import Postgres 


def geocode_and_insert(search_string, place_name, chunk_id):

    pg = Postgres()
    gmaps = googlemaps.Client(key=os.environ["GOOGLEMAPS_API_KEY"])
    geocode_result = gmaps.geocode(search_string)

    if geocode_result:
        # Extract the latitude, longitude, and formatted address
        location = geocode_result[0]['geometry']['location']
        lat = location['lat']
        lng = location['lng']
        formatted_address = geocode_result[0]['formatted_address']
        
        # Step 2: Insert the data into the PostgreSQL table using pg.query
        try:
            # Create the SQL query for inserting the data
            sql_query = """
                INSERT INTO locations (lat, lng, formatted_address, location_name, chunk_id)
                VALUES (%s, %s, %s, %s, %s)
            """
            
            # Execute the query and insert the geocoded data
            pg.insert(sql_query, (lat, lng, formatted_address, place_name, chunk_id))
            
            print(f"Inserted {place_name} into the locations table.")
            return True
        except Exception as e:
            print(f"Error inserting data into PostgreSQL: {e}")
            return False
    else:
        print(f"Error geocoding {place_name}: No results found")
        return False


In [4]:
from postgres import Postgres 
import json
pg = Postgres()

records = pg.query("""
    SELECT id, cmetadata->>'openai_geo_labels', cmetadata->>'context_text'
    FROM langchain_pg_embedding
    WHERE (cmetadata ? 'geocoding_complete') 
    AND cmetadata->>'experiment' = 'leeds'
    LIMIT 1000;
""")

for record in records:
   
    for openai_geo_label in json.loads(record[1]):
        print(openai_geo_label)
        
        geocode_and_insert("Leeds " + openai_geo_label, openai_geo_label, record[0])

query = """
    UPDATE langchain_pg_embedding
    SET cmetadata = jsonb_set(cmetadata, '{geocoding_complete}', 'true'::jsonb)
    WHERE id = %s
    """

pg.insert(query, (record[0],))        


Aberford
Inserted Aberford into the locations table.
Leeds
Inserted Leeds into the locations table.
A1(M) motorway
Inserted A1(M) motorway into the locations table.
Aberford Dykes/Becca Banks
Inserted Aberford Dykes/Becca Banks into the locations table.
Parlington
Inserted Parlington into the locations table.
St Ricarius Church
Error geocoding St Ricarius Church: No results found
Cock Beck
Inserted Cock Beck into the locations table.
Aberford House
Inserted Aberford House into the locations table.
Swan Hotel
Inserted Swan Hotel into the locations table.
Priory Park
Inserted Priory Park into the locations table.
The Great North Road
Inserted The Great North Road into the locations table.
Pump Hill village green
Inserted Pump Hill village green into the locations table.
Cattle Lane
Inserted Cattle Lane into the locations table.
Lotherton Lane
Inserted Lotherton Lane into the locations table.
Holbeck Moor
Inserted Holbeck Moor into the locations table.
Stocks Hill
Inserted Stocks Hill int